<a href="https://colab.research.google.com/github/Takumi173/DifyApps/blob/main/Dataset_JSON_Reviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## 処理用にデータを結合

In [1]:
# データのコピー
!git clone https://github.com/cdisc-org/sdtm-adam-pilot-project.git

Cloning into 'sdtm-adam-pilot-project'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 224 (delta 64), reused 220 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (224/224), 24.51 MiB | 2.54 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Updating files: 100% (87/87), done.


In [2]:
# 使用するjsonデータとdefine.xmlを新規ディレクトリにコピーする

import os
import shutil
import json

source_dir = "sdtm-adam-pilot-project/updated-pilot-submission-package/900172/m5/datasets/cdiscpilot01/tabulations/sdtm"
json_dir   = "json_files"
define_dir = "define_xml"

if not os.path.exists(json_dir):
    os.makedirs(json_dir)

if not os.path.exists(define_dir):
    os.makedirs(define_dir)

for root, _, files in os.walk(source_dir):
  for file in files:
    if file.endswith(".json"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(json_dir, file)
      shutil.copy(source_path, target_path)
    if file.endswith("define.xml"):
      source_path = os.path.join(root, file)
      target_path = os.path.join(define_dir, file)
      shutil.copy(source_path, target_path)

In [3]:
# jsonファイルをリスト形式に結合したファイル（dataset_list.json）を作成

dataset_list = []
for filename in os.listdir(json_dir):
  if filename.endswith(".json"):
    with open(os.path.join(json_dir, filename), "r") as f:
      try:
        json_data = json.load(f)
        dataset_list.append(json_data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {filename}: {e}")

with open("dataset_list.json", "w") as f:
  json.dump(dataset_list, f)


## 症例フィルタリング関数の定義

In [4]:
def filter_data(data, target_usubjids):
    """
    複数のドメインデータを含むリストから、指定されたUSUBJIDのrowsのみを抽出して新しいJSONファイルに保存する。
    入力データがリストでない場合はエラーメッセージを出力する。
    データ構造は、"columns" 内の "name" が "USUBJID" の列を持つことを前提とする。

    Args:
        data (list): ドメインを結合させたのリスト。リストでない場合はエラーとなる。
        output_file (str): 出力するJSONファイル名。
        target_usubjids (list): 残したいUSUBJIDのリスト。
    """
    if not isinstance(data, list):
        print("エラー：入力データはJSONオブジェクトのリストである必要があります。")
        return

    filtered_data_list = []
    for item in data:
        usubjid_index = -1
        if 'columns' in item:
            for i, col in enumerate(item['columns']):
                if 'name' in col and col['name'] == 'USUBJID':
                    usubjid_index = i
                    break

        if usubjid_index == -1:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'name' が 'USUBJID' の列が見つかりません。スキップします。")
            filtered_data_list.append(item)
            continue

        if 'rows' in item:
            filtered_rows = [
                row for row in item['rows'] if len(row) > usubjid_index and row[usubjid_index] in target_usubjids
            ]
            new_data = item.copy()
            new_data['rows'] = filtered_rows
            new_data['records'] = len(filtered_rows)
            filtered_data_list.append(new_data)
        else:
            print(f"警告：データセット '{item.get('fileOID', '不明')}' に 'rows' が見つかりません。スキップします。")
            filtered_data_list.append(item)

    return filtered_data_list



# 実行テスト
# with open('dataset_list.json', 'r') as f:
#   data = json.load(f)
#
# target_ids = ['01-701-1211']
# output_filename = 'filtered_list.json'
#
# filtered_data_list = filter_data(data, target_ids)
#
# with open(output_filename, 'w') as f:
#   json.dump(filtered_data_list, f)
#
# print(f"処理完了：'{output_filename}' に USUBJID が {target_ids} のデータを出力しました。")

## データ書き換え関数の定義

In [5]:
def data_update(data, target_domain, target_usubjid, target_seq, target_variable, new_value):
    """
    指定されたUSUBJIDを持つレコードの指定された変数を書き換えます。
    {target_domain}SEQが存在する場合はそれもキーとして使用します。
    元のデータは変更せず、新しいデータ構造を返します。

    Args:
        data (list): データ全体のリスト。指定されたtarget_domainのデータセットを含むことを想定します。
        target_domain (str): 対象のドメイン名（例: "CM"）。
        target_usubjid (str): 書き換えたいレコードのUSUBJID。
        target_seq (int): 書き換えたいレコードの{target_domain}SEQの値（存在しない場合は無視されます）。
        target_variable (str): 書き換えたい変数の名前（例: "CMTRT"）。
        new_value (any): 新しい変数の値。

    Returns:
        list: 指定された変数が更新された新しいデータ全体のリスト。
              該当するレコードが見つからなかった場合、元のデータのコピーを返します。
    """
    updated_data = []
    seqname = target_domain + 'SEQ'

    for dataset in data:
        updated_dataset = dataset.copy()
        if updated_dataset.get("itemGroupOID") == target_domain:
            updated_rows = []
            found = False
            usubjid_index = -1
            seq_index = -1
            variable_index = -1
            has_seq = False

            for i, col in enumerate(updated_dataset["columns"]):
                if col["name"] == "USUBJID":
                    usubjid_index = i
                elif col["name"] == seqname:
                    seq_index = i
                    has_seq = True
                elif col["name"] == target_variable:
                    variable_index = i

            if usubjid_index != -1 and variable_index != -1:
                for row in dataset["rows"]:
                    updated_row = list(row)  # 行をコピーして変更
                    usubjid_match = updated_row[usubjid_index] == target_usubjid
                    seq_match = True
                    if has_seq and seq_index != -1:
                        seq_match = (len(updated_row) > seq_index and updated_row[seq_index] == target_seq)
                    elif has_seq:
                        print(f"警告: '{target_domain}' データセットに '{seqname}' 列が見つかりましたが、インデックスが無効です。USUBJIDのみをキーとして使用します。")

                    if usubjid_match and seq_match:
                        updated_row[variable_index] = new_value
                        if has_seq and seq_index != -1:
                            print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' の '{target_variable}' を '{new_value}' に更新しました。")
                        else:
                            print(f"USUBJID '{target_usubjid}' の '{target_variable}' を '{new_value}' に更新しました。")
                        found = True
                    updated_rows.append(updated_row)
                updated_dataset["rows"] = updated_rows
            elif updated_dataset.get("itemGroupOID") == target_domain:
                print(f"'{target_domain}' データセットに 'USUBJID' または '{target_variable}' 列が見つかりませんでした。")

            updated_data.append(updated_dataset)
            if not found and updated_dataset.get("itemGroupOID") == target_domain:
                if has_seq and seq_index != -1:
                    print(f"USUBJID '{target_usubjid}'、'{seqname}' '{target_seq}' に該当するレコードが見つかりませんでした。")
                else:
                    print(f"USUBJID '{target_usubjid}' に該当するレコードが見つかりませんでした。")
        else:
            updated_data.append(updated_dataset)

    if not any(d.get("itemGroupOID") == target_domain for d in data):
        print(f"{target_domain} データセットが見つかりませんでした。")

    return updated_data

# 書き換えテスト
# updated_data = data_update(filtered_data_list, "DM", "01-701-1211", 0, "AGE", 49)
# updated_data = data_update(updated_data, "CM", "01-701-1211", 3, "CMTRT", "New Drug 123456789")
# updated_data = data_update(updated_data, "CM", "01-701-1211", 0, "CMDOSE", 123)

## データ比較関数の定義

In [6]:
from typing import List, Dict, Any

def compare_data(old_data: List[Dict[str, Any]], new_data: List[Dict[str, Any]]) -> None:
    """
    2つのデータリストの更新差分を人間が読みやすい形式で出力します。

    Args:
        old_data: 旧データリスト。
        new_data: 新データリスト。
    """

    def create_row_dict(item_group: Dict[str, Any], row: List[Any]) -> Dict[str, Any]:
        """rowデータをキー付きの辞書に変換する"""
        row_dict = {}
        for i, column in enumerate(item_group['columns']):
            row_dict[column['name']] = row[i]
        return row_dict

    def get_key_values(item_group_oid: str, row_dict: Dict[str, Any]) -> Dict[str, Any]:
        """データのキーとなる値を抽出する"""
        key_values = {'USUBJID': row_dict.get('USUBJID')}
        seq_key = f"{item_group_oid}SEQ"
        if seq_key in row_dict:
            key_values[seq_key] = row_dict[seq_key]
        return key_values

    def format_key(key_values: Dict[str, Any]) -> str:
        """キー値を人間が読みやすい文字列に整形する"""
        parts = []
        for key, value in key_values.items():
            if value is not None:
                parts.append(f"{key} = {value}")
        return ", ".join(parts)

    old_data_by_group = {item['itemGroupOID']: item for item in old_data}
    new_data_by_group = {item['itemGroupOID']: item for item in new_data}

    all_group_oids = set(old_data_by_group.keys()) | set(new_data_by_group.keys())

    for group_oid in sorted(list(all_group_oids)):
        print(f"--- ItemGroupOID: {group_oid} ---")
        old_group = old_data_by_group.get(group_oid)
        new_group = new_data_by_group.get(group_oid)

        old_rows_by_key = {}
        if old_group and 'rows' in old_group:
            for row in old_group['rows']:
                row_dict = create_row_dict(old_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    old_rows_by_key[format_key(key_values)] = row_dict

        new_rows_by_key = {}
        if new_group and 'rows' in new_group:
            for row in new_group['rows']:
                row_dict = create_row_dict(new_group, row)
                if 'USUBJID' in row_dict and row_dict['USUBJID'] is not None:
                    key_values = get_key_values(group_oid, row_dict)
                    new_rows_by_key[format_key(key_values)] = row_dict

        old_keys = set(old_rows_by_key.keys())
        new_keys = set(new_rows_by_key.keys())

        # 追加されたデータ
        added_keys = new_keys - old_keys
        for key in sorted(list(added_keys)):
            print(f"{key}:")
            print("  Added")
            for item_key, old_value in sorted(new_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 削除されたデータ
        removed_keys = old_keys - new_keys
        for key in sorted(list(removed_keys)):
            print(f"{key}:")
            print("  Deleted")
            for item_key, old_value in sorted(old_rows_by_key[key].items()):
                print(f"    {item_key}: {old_value}")
            print()

        # 更新されたデータ
        common_keys = old_keys & new_keys
        for key in sorted(list(common_keys)):
            if old_rows_by_key[key] != new_rows_by_key[key]:
                print(f"{key}:")
                print("  Updated:")
                old_row = old_rows_by_key[key]
                new_row = new_rows_by_key[key]
                for item_key in sorted(list(set(old_row.keys()) | set(new_row.keys()))):
                    old_value = old_row.get(item_key)
                    new_value = new_row.get(item_key)
                    if old_value != new_value:
                        print(f"    {item_key}: {old_value!r} -> {new_value!r}")
                print()


# 比較テスト
# compare_data(filtered_data_list, updated_data)

In [7]:
import json

usubjids = set()
for dataset in dataset_list:
    if 'columns' in dataset:
        for i, col in enumerate(dataset['columns']):
            if 'name' in col and col['name'] == 'USUBJID':
                if 'rows' in dataset:
                    for row in dataset['rows']:
                        if len(row) > i:
                            usubjids.add(row[i])

print(list(usubjids))


['01-708-1184', '01-716-1094', '01-701-1188', '01-709-1306', '01-713-1269', '01-716-1331', '01-704-1120', '01-708-1296', '01-701-1234', '01-704-1164', '01-717-1201', '01-716-1026', '01-701-1145', '01-701-1176', '01-701-1240', '01-705-1011', '01-711-1284', '01-715-1319', '01-705-1382', '01-703-1279', '01-709-1081', '01-708-1054', '01-708-1286', '01-703-1119', '01-708-1213', '01-718-1172', '01-701-1386', '01-708-1272', '01-701-1111', '01-703-1295', '01-701-1444', '01-715-1407', '01-709-1217', '01-701-1302', '01-709-1339', '01-707-1206', '01-703-1100', '01-704-1065', '01-710-1060', '01-703-1299', '01-709-1238', '01-715-1134', '01-716-1229', '01-703-1182', '01-718-1371', '01-704-1241', '01-707-1037', '01-708-1178', '01-710-1149', '01-701-1383', '01-701-1294', '01-704-1218', '01-708-1084', '01-709-1259', '01-716-1308', '01-708-1067', '01-703-1258', '01-710-1271', '01-710-1358', '01-710-1166', '01-701-1180', '01-711-1012', '01-710-1142', '01-703-1396', '01-715-1321', '01-709-1326', '01-707-1

# データの書き換え

In [8]:
Target_data = [
["DM", "01-703-1096",   0, "AGE", 49],
["LB", "01-703-1042",   3, "LBORRES", "135"],
["LB", "01-703-1042",   4, "LBORRES", "145"],
["LB", "01-703-1086",  37, "LBORRES", "1"],
["LB", "01-703-1086",  72, "LBORRES", "1.2"],
["LB", "01-703-1086", 102, "LBORRES", "1.1"],
["LB", "01-703-1086", 132, "LBORRES", "1"],
["LB", "01-703-1086", 162, "LBORRES", "1.3"],
["LB", "01-703-1086", 197, "LBORRES", "0.9"],
["LB", "01-703-1086", 232, "LBORRES", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESC", "135"],
["LB", "01-703-1042",   4, "LBSTRESC", "145"],
["LB", "01-703-1086",  37, "LBSTRESC", "1"],
["LB", "01-703-1086",  72, "LBSTRESC", "1.2"],
["LB", "01-703-1086", 102, "LBSTRESC", "1.1"],
["LB", "01-703-1086", 132, "LBSTRESC", "1"],
["LB", "01-703-1086", 162, "LBSTRESC", "1.3"],
["LB", "01-703-1086", 197, "LBSTRESC", "0.9"],
["LB", "01-703-1086", 232, "LBSTRESC", "0.8"],
["LB", "01-703-1042",   3, "LBSTRESN", 135],
["LB", "01-703-1042",   4, "LBSTRESN", 145],
["LB", "01-703-1086",  37, "LBSTRESN", 1],
["LB", "01-703-1086",  72, "LBSTRESN", 1.2],
["LB", "01-703-1086", 102, "LBSTRESN", 1.1],
["LB", "01-703-1086", 132, "LBSTRESN", 1],
["LB", "01-703-1086", 162, "LBSTRESN", 1.3],
["LB", "01-703-1086", 197, "LBSTRESN", 0.9],
["LB", "01-703-1086", 232, "LBSTRESN", 0.8],
["LB", "01-703-1042",   3, "LBNRIND", "HIGH"],
["LB", "01-703-1042",   4, "LBNRIND", "HIGH"],
["LB", "01-703-1086",  37, "LBNRIND", "LOW"],
["LB", "01-703-1086",  72, "LBNRIND", "LOW"],
["LB", "01-703-1086", 102, "LBNRIND", "LOW"],
["LB", "01-703-1086", 132, "LBNRIND", "LOW"],
["LB", "01-703-1086", 162, "LBNRIND", "LOW"],
["LB", "01-703-1086", 197, "LBNRIND", "LOW"],
["LB", "01-703-1086", 232, "LBNRIND", "LOW"],
["MH", "01-701-1097",   1, "MHTERM", "Loss of consciousness (Passed out)"],
["MH", "01-701-1097",   1, "MHSTDTC", "2023-01-01"],
["MH", "01-701-1111",   1, "MHTERM", "HEARING LOSS"],
["MH", "01-701-1180",   1, "MHTERM", "DEPRESSION (ANXIETY)"],
["MH", "01-702-1082",   1, "MHTERM", "Premenstrual pain"],
["MH", "01-703-1076",   1, "MHTERM", "Atrioventricular block (scheduled cardiac pacemaker insertion)"],
["MH", "01-703-1279",   1, "MHTERM", "schizophreniform disorders"],
["MH", "01-703-1299",   1, "MHTERM", "Cyclothymic disorder"],
["VS", "01-701-1047",  17, "VSORRES", "121"],
["VS", "01-701-1047",  18, "VSORRES", "124"],
["VS", "01-701-1047",  66, "VSORRES", "185"],
["VS", "01-701-1047",  67, "VSORRES", "183"],
["VS", "01-701-1383",  37, "VSORRES", "98"],
["VS", "01-701-1383", 122, "VSORRES", "160"],
["VS", "01-701-1387",   1, "VSORRES", "146"],
["VS", "01-701-1387",  32, "VSORRES", "72"],
["VS", "01-701-1047",  17, "VSSTRESC", "121"],
["VS", "01-701-1047",  18, "VSSTRESC", "124"],
["VS", "01-701-1047",  66, "VSSTRESC", "185"],
["VS", "01-701-1047",  67, "VSSTRESC", "183"],
["VS", "01-701-1383",  37, "VSSTRESC", "98"],
["VS", "01-701-1383", 122, "VSSTRESC", "160"],
["VS", "01-701-1387",   1, "VSSTRESC", "146"],
["VS", "01-701-1387",  32, "VSSTRESC", "72"],
["VS", "01-701-1047",  17, "VSSTRESN", 121],
["VS", "01-701-1047",  18, "VSSTRESN", 124],
["VS", "01-701-1047",  66, "VSSTRESN", 185],
["VS", "01-701-1047",  67, "VSSTRESN", 183],
["VS", "01-701-1383",  37, "VSSTRESN", 98],
["VS", "01-701-1383", 122, "VSSTRESN", 160],
["VS", "01-701-1387",   1, "VSSTRESN", 146],
["VS", "01-701-1387",  32, "VSSTRESN", 72],
["EX", "01-701-1148",   2, "EXDOSE", 82],
["EX", "01-701-1148",   3, "EXDOSE", 216],
["EX", "01-703-1258",   2, "EXDOSE", 27],
["CM", "01-701-1146",  29, "CMTRT", "PAROXETINE"],
["QS", "01-701-1023",1010, "QSORRES", "PRESENT"],
["QS", "01-701-1023",1012, "QSORRES", "PRESENT"],
["QS", "01-701-1111",5004, "QSORRES", "4"],
["QS", "01-701-1111",5019, "QSORRES", "4"],
["QS", "01-701-1111",5012, "QSORRES", "4"],
["QS", "01-701-1111",5027, "QSORRES", "4"],
["QS", "01-701-1118",6002, "QSORRES", "MARKED IMPROVEMENT"],
["QS", "01-701-1118",6003, "QSORRES", "MARKED WORSENING"],
["QS", "01-701-1181",4018, "QSORRES", "Y"],
["QS", "01-701-1181",4058, "QSORRES", "Y"],
["QS", "01-701-1181",4019, "QSORRES", "Y"],
["QS", "01-701-1181",4059, "QSORRES", "Y"],
["QS", "01-701-1181",4020, "QSORRES", "Y"],
["QS", "01-701-1023",1010, "QSSTRESC", "2"],
["QS", "01-701-1023",1012, "QSSTRESC", "2"],
["QS", "01-701-1111",5004, "QSSTRESC", "4"],
["QS", "01-701-1111",5019, "QSSTRESC", "4"],
["QS", "01-701-1111",5012, "QSSTRESC", "4"],
["QS", "01-701-1111",5027, "QSSTRESC", "4"],
["QS", "01-701-1118",6002, "QSSTRESC", "1"],
["QS", "01-701-1118",6003, "QSSTRESC", "7"],
["QS", "01-701-1181",4018, "QSSTRESC", "1"],
["QS", "01-701-1181",4058, "QSSTRESC", "1"],
["QS", "01-701-1181",4019, "QSSTRESC", "1"],
["QS", "01-701-1181",4059, "QSSTRESC", "1"],
["QS", "01-701-1181",4020, "QSSTRESC", "1"],
["QS", "01-701-1023",1010, "QSSTRESN", 2],
["QS", "01-701-1023",1012, "QSSTRESN", 2],
["QS", "01-701-1111",5004, "QSSTRESN", 4],
["QS", "01-701-1111",5019, "QSSTRESN", 4],
["QS", "01-701-1111",5012, "QSSTRESN", 4],
["QS", "01-701-1111",5027, "QSSTRESN", 4],
["QS", "01-701-1118",6002, "QSSTRESN", 1],
["QS", "01-701-1118",6003, "QSSTRESN", 7],
["QS", "01-701-1181",4018, "QSSTRESN", 1],
["QS", "01-701-1181",4058, "QSSTRESN", 1],
["QS", "01-701-1181",4019, "QSSTRESN", 1],
["QS", "01-701-1181",4059, "QSSTRESN", 1],
["QS", "01-701-1181",4020, "QSSTRESN", 1],
["QS", "01-701-1118",6001, "QSDTC", "2014-07-08"],
["QS", "01-701-1118",6001, "QSDY", 119],
["AE", "01-701-1015",   3, "AESER", "Y"],
["AE", "01-701-1015",   3, "AESHOSP", "Y"],
["AE", "01-701-1015",   3, "AESTDTC", "2014-01-11"],
["AE", "01-701-1015",   3, "AEENDTC", "2014-01-09"],
["AE", "01-701-1015",   3, "AESTDY", 10],
["AE", "01-701-1015",   3, "AEENDY", 8],
["AE", "01-701-1028",   1, "AETERM", "PARKINSON'S DISEASE"],
["AE", "01-701-1028",   1, "AESTDTC", "2013-07-01"],
["AE", "01-701-1028",   1, "AESTDY", -17],
["AE", "01-701-1034",   2, "AETERM", "MALIGNANT HYPERTENSION"],
["AE", "01-701-1047",   4, "AETERM", "HYPERTENSION"],
["AE", "01-701-1363",   1, "AESTDTC", "2013-06-15"],
["AE", "01-701-1363",   1, "AEENDTC", "2013-06-14"],
["AE", "01-701-1363",   1, "AESTDY", 17],
["AE", "01-701-1363",   1, "AEENDY", 16],
["AE", "01-701-1047",   3, "AEENDTC", "2013-03-05"],
["AE", "01-701-1047",   3, "AEENDY", 22],
["AE", "01-701-1383",  12, "AETERM", "BLOOD PRESSURE INCREASED"],
["AE", "01-701-1153",   2, "AEACN", "DRUG WITHDRAWN"],
["AE", "01-701-1180",   6, "AETERM", "SUDDEN DEATH"],
["AE", "01-703-1258",   2, "AESEV", "SEVERE"],
["AE", "01-703-1258",   2, "AESTDTC", "2012-08-01"],
["AE", "01-703-1258",   2, "AEENDTC", "2012-10-01"],
["AE", "01-703-1258",   2, "AESTDY", 13],
["AE", "01-703-1258",   2, "AEENDY", 74],
["AE", "01-703-1258",   5, "AESEV", "MODERATE"],
["AE", "01-703-1258",   5, "AESER", "Y"],
["AE", "01-703-1258",   5, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-703-1258",   5, "AESLIFE", "Y"],
["AE", "01-703-1258",   5, "AESTDTC", "2012-10-02"],
["AE", "01-703-1258",   5, "AEENDTC", "2012-12-31"],
["AE", "01-703-1258",   2, "AESTDY", 75],
["AE", "01-703-1258",   2, "AEENDY", 165],
["AE", "01-703-1335",   1, "AETERM", "MULTIPLE SCLEROSIS RELAPSE"],
["AE", "01-703-1335",   1, "AESTDTC", "2014-04-01"],
["AE", "01-703-1335",   1, "AEENDTC", "2014-05-01"],
["AE", "01-703-1335",   1, "AESTDY", 15],
["AE", "01-703-1335",   1, "AEENDY", 46],
["AE", "01-703-1403",   2, "AETERM", "MYASTHENIA GRAVIS AGGRAVATED"],
["AE", "01-704-1008",   1, "AETERM", "TREMOR IN HANDS, LEGS"],
["AE", "01-704-1008",   1, "AEREL", "NONE"],
["AE", "01-704-1008",   1, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   1, "AESTDY", -225],
["AE", "01-704-1008",   3, "AETERM", "MUSCLE STIFFNESS"],
["AE", "01-704-1008",   3, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   3, "AESTDY", -225],
["AE", "01-704-1008",   2, "AETERM", "SLOWNESS of MOVEMENT"],
["AE", "01-704-1008",   2, "AESTDTC", "2012-06-01"],
["AE", "01-704-1008",   2, "AESTDY", -225],
["AE", "01-704-1009",   6, "AETERM", "CHRONIC KIDNEY DISEASE"],
["AE", "01-704-1009",   6, "AESER", "Y"],
["AE", "01-704-1009",   6, "AESLIFE", "Y"],
["AE", "01-704-1010",   1, "AETERM", "DIABETES MELLITUS"],
["AE", "01-704-1010",   1, "AESER", "Y"],
["AE", "01-704-1010",   1, "AESLIFE", "Y"],
["AE", "01-704-1017",   4, "AETERM", "LATE EFFECTS OF CEREBRAL INFRACTION"],
["AE", "01-704-1017",   4, "AESEV", "SEVERE",],
["AE", "01-704-1017",   4, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   4, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   4, "AESTDY", 14],
["AE", "01-704-1017",   4, "AEENDY", 44],
["AE", "01-704-1017",   3, "AETERM", "BRAIN DEATH"],
["AE", "01-704-1017",   3, "AESEV", "SEVERE",],
["AE", "01-704-1017",   3, "AESTDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AEENDTC", "2013-11-18"],
["AE", "01-704-1017",   3, "AESTDY", 44],
["AE", "01-704-1017",   3, "AEENDY", 44],
["AE", "01-704-1017",   1, "AEOUT", "RECOVERED/RESOLVED"],
["AE", "01-704-1017",   1, "AESTDTC", "2013-10-19"],
["AE", "01-704-1017",   1, "AEENDTC", "2013-11-19"],
["AE", "01-704-1017",   1, "AESTDY", 14],
["AE", "01-704-1017",   1, "AEENDY", 45],
["AE", "01-704-1017",   1, "AEACN", "DRUG WITHDRAWN"]
]

dataset_list_updated = dataset_list

for l in Target_data:
  #print(l)
  dataset_list_updated = data_update(dataset_list_updated, l[0], l[1], l[2], l[3], l[4])

USUBJID '01-703-1096' の 'AGE' を '49' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBORRES' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBORRES' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBORRES' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBORRES' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBORRES' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '162' の 'LBORRES' を '1.3' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '197' の 'LBORRES' を '0.9' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '232' の 'LBORRES' を '0.8' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '3' の 'LBSTRESC' を '135' に更新しました。
USUBJID '01-703-1042'、'LBSEQ' '4' の 'LBSTRESC' を '145' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '37' の 'LBSTRESC' を '1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '72' の 'LBSTRESC' を '1.2' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '102' の 'LBSTRESC' を '1.1' に更新しました。
USUBJID '01-703-1086'、'LBSEQ' '132' の 'LBSTRESC' を '1' に更

In [9]:
# 更新データ確認
compare_data(dataset_list, dataset_list_updated)

with open("dataset_list_updated.json", "w") as f:
  json.dump(dataset_list_updated, f)

--- ItemGroupOID: AE ---
USUBJID = 01-701-1015, AESEQ = 3:
  Updated:
    AEENDTC: '2014-01-11' -> '2014-01-09'
    AEENDY: 10 -> 8
    AESER: 'N' -> 'Y'
    AESHOSP: 'N' -> 'Y'
    AESTDTC: '2014-01-09' -> '2014-01-11'
    AESTDY: 8 -> 10

USUBJID = 01-701-1028, AESEQ = 1:
  Updated:
    AESTDTC: '2013-07-21' -> '2013-07-01'
    AESTDY: 3 -> -17
    AETERM: 'APPLICATION SITE ERYTHEMA' -> "PARKINSON'S DISEASE"

USUBJID = 01-701-1034, AESEQ = 2:
  Updated:
    AETERM: 'FATIGUE' -> 'MALIGNANT HYPERTENSION'

USUBJID = 01-701-1047, AESEQ = 3:
  Updated:
    AEENDTC: '' -> '2013-03-05'
    AEENDY: None -> 22

USUBJID = 01-701-1047, AESEQ = 4:
  Updated:
    AETERM: 'BUNDLE BRANCH BLOCK LEFT' -> 'HYPERTENSION'

USUBJID = 01-701-1153, AESEQ = 2:
  Updated:
    AEACN: '' -> 'DRUG WITHDRAWN'

USUBJID = 01-701-1180, AESEQ = 6:
  Updated:
    AETERM: 'MICTURITION URGENCY' -> 'SUDDEN DEATH'

USUBJID = 01-701-1363, AESEQ = 1:
  Updated:
    AEENDTC: '2013-06-15' -> '2013-06-14'
    AEENDY: 17 -> 16

# LLMへの送信

In [18]:
!pip install sseclient-py
import requests
import sseclient
from IPython.display import display, Markdown

from google.colab import userdata
api_key = userdata.get('Dify_DatasetJSON')
user_id = 'JPMA_Sample'

## 関数定義

In [19]:
import json
import time
import requests
import sseclient

# 定数
DIFY_API_URL = 'https://api.dify.ai/v1/workflows/run'
CONTENT_TYPE_JSON = 'application/json'

def call_dify_api(api_key: str, payload: dict, stream: bool = False) -> requests.Response:
    """Dify APIを呼び出す共通関数"""
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': CONTENT_TYPE_JSON
    }
    try:
        response = requests.post(DIFY_API_URL, headers=headers, json=payload, stream=stream)
        response.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる
        return response
    except requests.exceptions.RequestException as e:
        print(f"API呼び出しエラー: {e}")
        raise

def run_dify_workflow(api_key: str, workflow_inputs: dict, user_id: str, streaming: bool = False) -> dict | sseclient.Event:
    """Difyワークフローを実行する

    Args:
        api_key: Dify APIキー
        workflow_inputs: ワークフローへの入力
        user_id: ユーザーID
        streaming: ストリーミングモードで実行するかどうか (Falseの場合はブロッキングモード)

    Returns:
        ストリーミングモードの場合はsseclient.Eventのイテレータ、
        ブロッキングモードの場合はAPIのレスポンスのJSONを辞書型で返す
    """
    payload = {
        'inputs': workflow_inputs,
        'response_mode': 'streaming' if streaming else 'blocking',
        'user': user_id
    }
    response = call_dify_api(api_key, payload, stream=streaming)
    if streaming:
        client = sseclient.SSEClient(response)
        return client.events()
    else:
        return response.json()

def safe_print_event_data(event: sseclient.Event):
    """
    与えられたSSEイベントデータから、存在する場合に特定の値を出力します。
    キーが存在しない場合は何も出力しません。Statusが存在する場合にのみTitleと結合させて表示します。

    Args:
        event: イベントデータを含むSSEイベントオブジェクト。event.data属性がJSON文字列であることを想定。
    """
    try:
        data = json.loads(event.data)

        if 'event' in data:
            print(f"Event: {data['event']}")

        if 'data' in data:
            title = data['data'].get('title')
            status = data['data'].get('status')
            if title is not None and status is not None:
                print(f"Node: {title} ({status})")
            elif title is not None:
                print(f"Node: {title}") # Statusが存在しない場合はTitleのみ表示

            if 'error' in data['data']:
                print(f"Error: {data['data']['error']}")
            if 'elapsed_time' in data['data']:
                print(f"Elapsed time: {data['data'].get('elapsed_time')}")
            if 'total_tokens' in data['data']:
                print(f"Total tokens: {data['data'].get('total_tokens')}")

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON event data: {e}")
    except AttributeError as e:
        print(f"Error accessing event data attribute: {e}")

def run_workflow_with_retry(api_key: str, workflow_inputs: dict, user_id: str, max_retries: int = 3, retry_delay: int = 20):
    """ワークフローを実行し、エラー発生時にリトライを行う (ストリーミングモード専用)"""
    for retry in range(max_retries + 1):
        print(f"--- 試行回数: {retry + 1} ---")
        success = True
        try:
            for event in run_dify_workflow(api_key, workflow_inputs, user_id, streaming=True):
                safe_print_event_data(event)
                try:
                    event_data = json.loads(event.data)
                    if event_data.get('data', {}).get('error') is not None:
                        print(f"エラーが検出されました: {event_data['data']['error']}")
                        success = False
                        break
                except json.JSONDecodeError:
                    print("JSONデコードエラーが発生しました。")
                    success = False
                    break
                print('------')

            if success:
                print("ワークフローが正常に完了しました。")
                return json.loads(event.data)
            elif retry < max_retries:
                print(f"エラーが発生したため、{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

        except requests.exceptions.RequestException as e:
            print(f"APIリクエスト中にエラーが発生しました: {e}")
            success = False
            if retry < max_retries:
                print(f"{retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("最大再試行回数に達しました。ワークフローは失敗しました。")
                return False

## プロンプトの作成

In [20]:
with open('define_xml/define.xml', 'r') as f:
  define_xml = f.read()


SysPrompt = '''
あなたは、臨床試験データのレビューを支援するAIアシスタントです。以下の前提知識を理解した上で、ユーザーからの指示（ユーザープロンプト）に従って、臨床試験データのレビューを支援してください。各タスクでは、ユーザープロンプトで指定された役割になりきって回答してください。

**前提知識:**

*   臨床試験においては患者の安全性が最優先され、有害事象の評価は特に重要です。
*   SDTM (Study Data Tabulation Model) は、CDISCによって策定された臨床試験データの標準モデルです。
*   Define.xmlはSDTMデータの構造を記述したメタデータファイルであり、参考情報として使用します。JSONデータ自体の内容、医学的妥当性、プロトコルとの整合性を優先してレビューしてください。
*   SDTMデータは、DM、AE、VS、LBなど、複数のドメイン（データセット）に分かれています。
*   報告されるJSONデータには、データ入力時の間違いが含まれる可能性があります。
*   提供された情報のみに基づいて回答を作成してください。想像やハルシネーションに基づいた回答は作成してはいけません。

**出力形式:**

*   すべての出力はMarkdown形式で作成してください。
*   見出し記号(#)は使用しないでください。

**その他:**

*   JSONデータまたはDefine.xmlの形式が不正な場合は、その旨をエラーメッセージとして出力してください。
'''




UserInput_Task1 = '''
あなたは臨床試験の専門医です。以下の指示に従い、提供される情報（プロトコル、JSONデータ、Define.xml）を基に、臨床試験データのレビューとクエリ作成（必要な場合）を行ってください。

**1. 症例サマリーの作成:**

*   **参照情報:** JSONデータ、Define.xml
*   **タスク:**
    *   JSONデータとDefine.xmlを参照し、有害事象、検査値、バイタルサインなどの推移を時系列でまとめた症例サマリーを作成してください。
    *   特に、**異常所見**を中心に簡潔な文章で記載してください。正常範囲内の変動は省略して構いません。
    *   各イベントの日時は、Define.xmlに定義された日付変数などを参考に、正確に特定してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **患者ID:**
        *   YYYY年MM月DD日 (Day XX): [有害事象、検査値、バイタルサインなどのイベントを、異常所見を中心に簡潔な文章で記載]

---

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   以下のJSONデータのレビュー観点に基づき、JSONデータを改めて点検してください。
    *   医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *  **疑義事項がない場合は、クエリを作成する必要はありません。**「疑義事項なし」と回答してください。

*   **JSONデータのレビュー観点 (これらに限定されない):**
    *   **安全性:** 有害事象(AEドメイン)の報告内容は、医学的に妥当であるか？
    *   **医学的妥当性:** 検査値(LBドメイン)の変動、バイタルサイン(VSドメイン)の変動、併用薬(CMドメイン)との相互作用など、時間経過とともに医学的に問題となる点は見られるか？
    *   **有効性:** 特定された主要評価項目および副次評価項目について、その時間的変化は期待される効果と一致しているか？
    *   **その他:** 患者背景(DMドメイン)、既往歴(MHドメイン)、有害事象(AEドメイン)、治療歴(EXドメイン, CMドメイン)などを総合的に考慮し、時間経過を加味して安全性に懸念を生じる事項があれば記載してください。
    *   **プロトコル逸脱 (疑い):** 選択/除外基準、投与量、併用禁止薬、評価スケジュール、有害事象報告などについて、プロトコルからの逸脱の疑いがないか確認してください。（関連ドメイン: DM, MH, EX, CM, LB, VS, AEなど）

*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''



UserInput_Task2 = '''
あなたはクリニカルデータマネージャーです。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、データ整合性レビューとクエリ作成（必要な場合）を行ってください。

**1. データ整合性レビュー:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、データの不整合が疑われる問題点を検出してください。
    *   **特に、以下の点に焦点を当ててレビューしてください。**
        *   **クロスドメイン整合性:** 異なるSDTMドメイン間で、データに矛盾がないか、ドメイン間の関連性が正しく表現されているか。
            *   **具体的な確認例 (これらに限定されない):**
                *   DM.SEXとAEにおける妊娠関連の有害事象
                *   AEの有害事象発現日や治験薬との関連性と、EXの治験薬の投与期間
                *   LBの検査値異常とAEの関連有害事象
                *   VSのバイタルサイン異常とAEの関連有害事象
                *   CM.CMTRTとAE/MHで報告されている疾患・既往歴との矛盾
        *   **単一ドメイン内の整合性:** Define.xmlの定義に照らして、矛盾なく解釈できるデータになっているか、プロトコルに照らしてデータの関連性が正しく表現されているか。
        *   **異常値:** Define.xmlで定義された範囲外、または医学的にありえない値がないか。
        *   **欠損値:** 欠損値の有無と理由（推測できる場合）。多い場合は原因を推測。
        *   **プロトコル逸脱 (データ品質の観点から):** データ入力/収集で、プロトコルからの逸脱（例：必須項目の未入力、不適切な時期のデータ収集）がないか。
    *   Define.xmlとデータの間に不整合がある場合は、「Define.xmlの修正候補」として報告してください。
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **全体的なデータ品質の評価:**
        *   総合評価: [例：良好、一部問題あり、要修正]
        *   データクリーニング/再調査が必要な項目: [該当項目を列挙]

    *   **問題点:**（問題がある場合）
        *   **問題No.:**
            *   **変数名と値:**
            *   **矛盾の内容:** [具体的な矛盾の内容を記述]
        *   **問題点の原因（推測）:**
        *   **対応策（提案）:**

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   データ整合性レビューの結果、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、臨床試験の評価項目に対する影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''


UserInput_Task3 = '''
あなたは、臨床試験の専門医、データマネージャー、CRAの視点を持つ、プロトコル遵守状況の確認者です。以下の指示に従い、提供される情報（JSONデータ、Define.xml、プロトコル）を基に、プロトコル逸脱の検出とクエリ作成（必要な場合）を行ってください。

**1. プロトコル逸脱の検出:**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   JSONデータ、Define.xml、プロトコルを参照し、プロトコルからの逸脱を検出してください。
    *   Define.xmlは参考情報として活用し、データとプロトコルの内容を比較して逸脱を判断してください。
    *   **検出対象とすべき主要なプロトコル逸脱の例 (これらに限定されない):**
        *   **選択/除外基準違反:** (関連SDTMドメイン: DM, MH など)
        *   **投与量違反:** (関連SDTMドメイン: EX)
        *   **併用禁止薬の使用:** (関連SDTMドメイン: CM)
        *   **評価スケジュール違反:** (関連SDTMドメイン: LB, VS, その他)
        *   **有害事象報告違反**: (関連SDTMドメイン: AE)
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。

    *   **患者ID:**
        *   **逸脱No.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor）
            *   **変数名と値:**
            *   **逸脱内容:** [具体的な逸脱内容を簡潔に記述。例：被験者XXXは、プロトコルで規定された投与量を超える量の治験薬を投与された]
            *   **プロトコル該当箇所:** [プロトコルの該当するセクション、ページ番号などを記載]
            *   **判断理由:**

**2. クエリの作成 (必要な場合のみ):**

*   **参照情報:** JSONデータ、Define.xml、プロトコル
*   **タスク:**
    *   プロトコル逸脱を判定するために、医療機関への問い合わせが必要な事項（疑義、不明点、確認事項など）が発生した場合、その内容をまとめたクエリを作成してください。
    *   クエリは、報告されたデータと、Define.xml、プロトコルの記述に基づいて作成してください。提供された情報から逸脱する内容や、想像、ハルシネーションに基づくクエリは作成してはいけません。
    *   クエリは、プロトコル逸脱が臨床試験の評価項目に与える影響度を考慮し、重要度の高いものから優先的に作成してください。
    *   **プロトコル逸脱に関する疑義事項がない場合は、クエリを作成する必要はありません。**
*   **出力形式:** 以下のテンプレートに従ってMarkdown形式で出力してください。クエリがない場合は、「クエリなし」と出力してください。

    *   **患者ID:**
        *   **クエリNo.:**
            *   **臨床試験結果への影響度合い:**（Critical/Major/Minor/None）
            *   **変数名と値:**
            *   **医療機関への問い合わせ文面:**
            *   **判断理由:**
'''


UserInput_end1 = '''\n---\n\n**データ:**\n\n*   臨床試験データ（JSON形式、SDTM準拠）:\n\n```json\n'''
UserInput_end2 = '''\n```\n\n*   データ定義ファイル（Define.xml）:\n\n```xml\n''' + define_xml + '''```\n'''

In [21]:
def create_workflow_input(ModelName, SysPrompt, UserInput_Task, datasetjson, UserInput_end1, UserInput_end2):
    return {
        'ModelName': ModelName,
        'SysPrompt': SysPrompt,
        'UserInput': UserInput_Task + UserInput_end1 + datasetjson + UserInput_end2,
        'AttachedFile': {"type": "document", "transfer_method": "local_file", "upload_file_id": "6b06d4f8-d47a-441f-bf67-d8700f76f556"}
    }

## 実行

In [22]:
# ModelNameの設定
#ModelName = 'gemini-2.0-flash'
#ModelName = 'gemini-2.0-flash-exp'
ModelName = 'gemini-2.0-flash-exp-multi'
#ModelName = 'gemini-2.0-pro-exp-02-05'
#ModelName = 'gemini-2.0-flash-thinking-exp-01-21'
#ModelName = 'gemini-2.0-flash-thinking-exp-01-21-multi'
#ModelName = 'gemini-2.0-flash-thinking-exp'
#ModelName = 'gemini-2.0-flash-thinking-exp-multi'


# データ更新症例の抽出
updated_subjects = []
for l in Target_data:
  updated_subjects.append(l[1])

updated_subjects = list(set(updated_subjects))
print(updated_subjects)


['01-701-1097', '01-701-1387', '01-701-1028', '01-704-1010', '01-701-1148', '01-701-1023', '01-703-1335', '01-703-1403', '01-701-1181', '01-701-1015', '01-703-1279', '01-701-1034', '01-704-1008', '01-701-1047', '01-701-1111', '01-701-1118', '01-703-1299', '01-703-1096', '01-701-1383', '01-701-1153', '01-703-1258', '01-704-1009', '01-701-1180', '01-703-1042', '01-702-1082', '01-703-1076', '01-701-1363', '01-703-1086', '01-704-1017', '01-701-1146']


In [24]:
import pandas as pd

# リトライ付きでワークフローを実行
results_list = []

for subj in updated_subjects:
    datasetjson = filter_data(dataset_list_updated, subj)
    print(f"処理完了：'datasetjson' に USUBJID が {subj} のデータを出力しました。")

    row_data = {'Subject': subj}  # 各行のデータを格納する辞書

    # Task 1 の処理
    workflow_inputs_Task1 = create_workflow_input(ModelName, SysPrompt, UserInput_Task1, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    try:
        result_Task1 = run_workflow_with_retry(api_key, workflow_inputs_Task1, user_id)
        output_Task1 = result_Task1['data']['outputs']['text']
        display(Markdown(output_Task1))
        row_data['Task1'] = output_Task1
    except Exception as e:
        print(f"Task 1 でエラーが発生しました (Subject: {subj}): {e}")
        row_data['Task1'] = "Error"

    # Task 2 の処理
    workflow_inputs_Task2 = create_workflow_input(ModelName, SysPrompt, UserInput_Task2, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    try:
        result_Task2 = run_workflow_with_retry(api_key, workflow_inputs_Task2, user_id)
        output_Task2 = result_Task2['data']['outputs']['text']
        display(Markdown(output_Task2))
        row_data['Task2'] = output_Task2
    except Exception as e:
        print(f"Task 2 でエラーが発生しました (Subject: {subj}): {e}")
        row_data['Task2'] = "Error"

    # Task 3 の処理
    workflow_inputs_Task3 = create_workflow_input(ModelName, SysPrompt, UserInput_Task3, UserInput_end1, json.dumps(datasetjson), UserInput_end2)
    try:
        result_Task3 = run_workflow_with_retry(api_key, workflow_inputs_Task3, user_id)
        output_Task3 = result_Task3['data']['outputs']['text']
        display(Markdown(output_Task3))
        row_data['Task3'] = output_Task3
    except Exception as e:
        print(f"Task 3 でエラーが発生しました (Subject: {subj}): {e}")
        row_data['Task3'] = "Error"

    results_list.append(row_data)

# DataFrameを作成
df_results = pd.DataFrame(results_list)

# DataFrameを表示
display(df_results)

警告：データセット 'CDISCPILOT01.ta' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.te' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ti' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.ts' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
警告：データセット 'CDISCPILOT01.tv' に 'name' が 'USUBJID' の列が見つかりません。スキップします。
処理完了：'datasetjson' に USUBJID が 01-701-1097 のデータを出力しました。
--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.053649
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.265403
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.165685
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (

## 症例サマリー

**患者ID:** 01-701-1097

*   2013年12月23日 (Day -9): スクリーニング検査で、ナトリウムが基準値より低い(134 mEq/L、基準範囲: 135-145 mEq/L)。
*   2014年02月20日 (Day 51): 適用部位小胞を発症。重症度は軽度で、2014年2月22日に回復。
*   2014年02月20日 (Day 51): 全身性掻痒症を発症。重症度は中等度で、2014年3月21日に回復。
*   2014年02月21日 (Day 52): 適用部位そう痒症が発生。重症度は軽度。
*   2014年02月26日 (Day 57): Week8の検査で、好酸球数が基準値より高い(0.89 THOU/uL, 基準値: 0-0.57 THOU/uL)。
*   2014年03月26日 (Day 85): Week12の検査で、好酸球数が基準値より高い(0.87 THOU/uL, 基準値: 0-0.57 THOU/uL)。
*   2014年04月19日 (Day 109): 鼻詰まりと咽喉頭痛を発症。重症度はいずれも軽度で、2014年4月22日に回復。
*   2014年05月21日 (Day 141): 収縮期血圧が低下(112 mmHg、臥位)。
*   2014年05月21日 (Day 141): 好酸球数が基準値内へ改善(0.44 THOU/uL、基準範囲: 0-0.57 THOU/uL)
*   2014年07月09日 (Day 190): 試験を完了。

---

**クエリ:**

患者ID: 01-701-1097

*   **クエリNo.:** 1
    *   **臨床試験結果への影響度合い:** Major
    *   **変数名と値:** AE.AESTDTC, AE.AEENDTC, AE.AEDTC, AE.AETERM
    *   **医療機関への問い合わせ文面:** 
        *   適用部位そう痒症の有害事象について、AESTDTCとAEDTCに矛盾が見られます。AESTDTCとAEDTCについて、正しい日付を教えてください。
        *   AEENDTCが空欄となっていますが、これは継続中の事象でよろしいでしょうか。
    *   **判断理由:** AEドメインのAESTDTC、AEENDTC、AEDTCは有害事象の期間を特定するために重要な変数です。不正確な日付は、有害事象の正確な期間を把握する妨げとなり、安全性評価に影響を与える可能性があります。
*   **クエリNo.:** 2
    *   **臨床試験結果への影響度合い:** Minor
    *   **変数名と値:** LB.LBTESTCD = "SODIUM", LB.LBCAT = "CHEMISTRY", LB.LBDTC = "2013-12-23T12:30", LB.LBORRES = 134, LB.LBORRESU = "mEq/L"
    *   **医療機関への問い合わせ文面:** スクリーニング時の血清ナトリウム値が基準範囲を下回っていますが、除外基準27b (Laboratory test values exceeding the Lilly Reference Range III for the patient’s age in any of the following analytes) に抵触しない理由を教えてください。臨床的に特筆すべき所見はありましたか？
    *   **判断理由:** 
        *   DM.RFSTDTC（2014-01-01）以前に発生した事象であること。
        *   治験薬投与開始前のデータに疑義がある場合、組み入れ基準を満たしているかどうかの判断に影響を及ぼす可能性があるため。
*   **クエリNo.:** 3
    *   **臨床試験結果への影響度合い:** Minor
    *   **変数名と値:** QS.QSTESTCD = "DAITM27", QS.QSORRES = "NA", VISIT = "WEEK 24", QS.QSTESTCD = "DAITM28", QS.QSORRES = "NA", VISIT = "WEEK 16", QS.QSTESTCD = "DAITM29", QS.QSORRES = "NA", VISIT = "WEEK 16"
    *   **医療機関への問い合わせ文面:** DADの質問項目DAITM27 (GO OUT AND REACH A FAMILIAR DESTINATION), DAITM28 (SAFELY TAKE CAR, BUS, TAXI), DAITM29 (RETURN FROM THE STORE)について、WEEK 16, WEEK 24でNAと回答されています。これは、元々実施していなかったのか、それとも実施を試みたができなかったのか、理由をご教示ください。
    *   **判断理由:** 評価が適用不能である理由を把握する必要があるため。
*   **クエリNo.:** 4
    *   **臨床試験結果への影響度合い:** Minor
    *   **変数名と値:** VS.VSTEST = "Temperature", VSORRES = "93.7", VISIT = "WEEK 4"
    *   **医療機関への問い合わせ文面:** WEEK4での体温が93.7Fと異常に低い値を示していますが、これは記録された値として正しいですか？患者に低体温症の症状は見られましたか？測定方法に誤りはありませんでしょうか。
    *   **判断理由:** 体温の異常値は、患者の安全性に関わる可能性があるため。
*   **クエリNo.:** 5
    *   **臨床試験結果への影響度合い:** Minor
    *   **変数名と値:** 全般
    *   **医療機関への問い合わせ文面:** NPI-XのスコアがBaseline, Week2, Week4, Week6, Week8, Week12, Week14(T), Week16, Week18(T), Week20, Week22(T), Week24, Week26で一貫して0である項目が複数あります。これらのスコアは本当に変化がないのか、確認してください。
    *   **判断理由:** NPI-Xは認知症患者の精神症状を評価するものであり、すべての期間で症状がないというのは不自然である可能性があるため。



--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.042847
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.220474
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.123584
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T: 0.5) gemini-2.0-flash-exp 
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T: 0.3) gemini-2.0-flash-exp
------
Event: node_finished
Node: (3 T: 0.7) gemini-2.0-flas

**全体的なデータ品質の評価:**
*   総合評価: 一部問題あり
*   データクリーニング/再調査が必要な項目: CM.CMTRT, CM.CMSTDTC, AE.AESTDTC, LB.LBDTC, DS.DSDTC, LB.LBORRES, LB.LBORRESU, QS.QSDTC

問題点:
*   **問題No.1:**
    *   **変数名と値:** CM.CMTRT = CENTRUM, CM.CMSTDTC = 2013-10
    *   **矛盾の内容:** CMSTDTC（併用薬開始日）がYYYY-MM形式で記録されており、CM.CMDTC（データ収集日）より早い。CMSTDTCの日付が正しくありません。
    *   **問題点の原因（推測）:** データ入力時の誤り、または情報源の記録不備。
    *   **対応策（提案）:** CM.CMTRTがCENTRUMであるすべてのレコードに対して、CMSTDTCの完全な日付（YYYY-MM-DD）を調査し、更新する。CMSTDTCがDM.DMDTCより前のレコードが存在しないか確認する。

*   **問題No.: 2:**
    *   **変数名と値:** AE.AESTDTC = 2014-02-21, AE.AEENDTC = ""
    *   **矛盾の内容:** AESTDTC（有害事象開始日）がAEENDTC（有害事象終了日）より後になっている、またはAEENDTCが空欄。
    *   **問題点の原因（推測）:** データ入力時の誤り、または有害事象が継続中。
    *   **対応策（提案）:** AE.AESTDTCがAEENDTCより後のレコード、またはAEENDTCが空欄のレコードについて、終了日を確認し、矛盾を解消する。AEENDTCに値が入るべきか確認し、必要であればAEOUTの値を更新する。

*   **問題No.: 3:**
    *   **変数名と値:** LB.LBDTCとLB.VISITDYの不整合
    *   **矛盾の内容:** LB.LBDTCの日付と、各検査値のVISITDYを比較すると、ずれがある箇所が複数存在する。例：LBSEQ=1のLBDTCは2013-12-23T12:30だが、VISITDYは-9。
    *   **問題点の原因（推測）:** LBDTCの日付入力ミス、またはVISITDYの計算ミス。
    *   **対応策（提案）:** LB.LBDTCの日付が、LB.VISITDY（計画された来院日）から期待される日付と大きく異なるレコードについて、LBDTCの日付が正しいかどうか確認する。LBDTCの日付がVISITDYから大きくずれている場合、その理由を調査する（例：検査の遅延、別のVisitで実施）。

*   **問題No.: 4:**
    *   **変数名と値:** DM.DMDTC = "2013-12-23", DM.DMDY = -9
    *   **矛盾の内容:** DMDTC（人口統計データ収集日）が2013-12-23だが、DMDY（試験日）が-9となっている。しかし、LBのデータを見ると、LBDYも-9であり、LBDTCの日付も2013-12-23である。
    *   **問題点の原因（推測）:** データ入力時の誤り、または参照開始日が誤っている可能性。
    *   **対応策（提案）:** DM.RFSTDTC（Subject Reference Start Date/Time）の値を確認し、DMDYが正しく計算されているか確認する。DM.RFSTDTCとLB.LBDTC、AE.AESTDTCなどの日付との関係性を確認し、矛盾がないか確認する。

*   **問題No.: 5:**
    *   **変数名と値:** VS.VSDTCとVSの各バイタルサイン
    *   **矛盾の内容:** VS.VSDTCの日付と、各バイタルサインのVISITDYを比較すると、ずれがある箇所が複数存在する。例：VSSEQ=1のVSDTCは2013-12-23だが、VISITDYは-9。
    *   **問題点の原因（推測）:** VSDTCまたはVISITDYのデータ入力エラー、またはVISITDYの導出ロジックの誤り
    *   対応策（提案）: VSDTCとVISITDYの整合性を検証し、必要に応じて修正する。

*   **問題No.: 6:**
    *   **変数名と値:** AE.AETERM = "APPLICATION SITE PRURITUS", CM.CMTRT = "HYDROCORTISONE"
    *   **矛盾の内容:** AETERMに「APPLICATION SITE PRURITUS（適用部位掻痒）」が報告されているが、CMTRTに「HYDROCORTISONE（ヒドロコルチゾン）」が併用薬として記録されている。
    *   **問題点の原因（推測）:** 適用部位掻痒に対する治療薬としてヒドロコルチゾンが使用された可能性があるが、CMドメインにindication（CMIND）が記録されていない。
    *   対応策（提案）: CMドメインにCMIND（indication）を追加し、ヒドロコルチゾン使用の理由を明確にする。

*   **問題No.: 7:**
    *   **変数名と値:** DM.SEX = "M", MH.MHTERM = "ALZHEIMER'S DISEASE"
    *   **矛盾の内容:** DM.SEXが「M（男性）」である被験者に、MHTERMに「ALZHEIMER'S DISEASE（アルツハイマー病）」の既往歴がある。TIドメインのInclusion Criteriaには「Males and postmenopausal females at least 50 years of age.（50歳以上の男性および閉経後の女性）」と記載されている。
    *   問題点の原因（推測）: データ入力エラー、またはプロトコル逸脱
    *   対応策（提案）: DM.SEXが「M（男性）」で、MHに「ALZHEIMER'S DISEASE」の病歴があることの医学的な妥当性を確認する。TIドメインのInclusion Criteriaとの整合性を確認する。

*   **問題No.: 8:**
    *   **変数名と値:** LB.LBORRES = "43", LB.LBORRESU = "g/L" (レコード1)
    *   **矛盾の内容:** 
        *   LB.LBORRESの値が数値であるにもかかわらず、データ型が文字列となっている。
        *   LB.LBSTRESNが43であるにもかかわらず、LB.LBORRESが4.3である。
    *   問題点の原因（推測）: データ入力時の単位の誤り、またはデータ変換の誤り。
    *   対応策（提案）:
        1.  LB.LBORRESの値が数値である場合、データ型を数値に修正する。
        2.  LB.LBORRESの値とLB.LBORRESUの値が整合性が取れているか確認する。
        3.  LB.LBSTRESNの値とLB.LBORRESの値が単位換算されているか確認する。

*   **問題No.: 9:**
    *   **変数名と値:** VISITの値に"WEEK 10 (T)"、"WEEK 14 (T)"、"WEEK 18 (T)"、"WEEK 22 (T)"
    *   **矛盾の内容:** VISITの値に"WEEK 10 (T)"、"WEEK 14 (T)"、"WEEK 18 (T)"、"WEEK 22 (T)"といった、WEEKと数字と(T)が組み合わさった値がVS, LB, QSドメインに存在しますが、TVドメインのVISITの値には存在しません。
    *   問題点の原因（推測）: TVドメインに定義されていないVISITの値が、VS, LB, QSドメインで使用されている。
    *   対応策（提案）: TVドメインに"WEEK 10 (T)"、"WEEK 14 (T)"、"WEEK 18 (T)"、"WEEK 22 (T)"というVISITの値を追加するか、VS, LB, QSドメインのVISITの値をTVドメインに存在する値に修正する。

*   **問題No.: 10:**
    *   **変数名と値:** Define.xmlにおけるCMDTC, AEDTC, LBDTC, QSDTCのDataType
    *   **矛盾の内容:** Define.xmlにおいて、CMDTC, AEDTC, LBDTC, QSDTCのDataTypeが"date"となっているが、ISO8601形式の日付時刻を表す"datetime"である方が適切である。
    *   問題点の原因（推測）: Define.xmlの定義が不適切。
    *   対応策（提案）: Define.xmlを修正し、CMDTC, AEDTC, LBDTC, QSDTCのDataTypeを"datetime"に変更する。

クエリ:
*   患者ID: 01-701-1097
    *   クエリNo.: 1
        *   臨床試験結果への影響度合い: Major
        *   変数名と値: CM.CMTRT = "CENTRUM", CM.CMSTDTC = "2013-10"
        *   医療機関への問い合わせ文面:
            *   CMTRTがCENTRUMである薬剤の正確な投与開始日を教えてください。
            *   CMSTDTCに日の情報がないため、特定をお願いします。
            *   もし正確な日付が不明な場合、可能な限り正確な日付、または最も可能性の高い日付をご教示ください。
        *   判断理由: 併用薬の使用期間は、試験結果に影響を与える可能性があるため。

    *   クエリNo.: 2
        *   臨床試験結果への影響度合い: Major
        *   変数名と値: AE.AESTDTC = "2014-02-21", AE.AEENDTC = ""
        *   医療機関への問い合わせ文面:
            *   AETERMがAPPLICATION SITE PRURITUSである有害事象の終了日を教えてください。
            *   AEENDTCが空欄ですが、有害事象は回復/解決しましたか？解決した場合、解決日を教えてください。
        *   判断理由: 有害事象の期間は重要な安全性情報であり、正確なデータが必要なため。

    *   クエリNo.: 3
        *   臨床試験結果への影響度合い: Minor
        *   変数名と値: LB.LBDTC, LB.VISITDY
        *   医療機関への問い合わせ文面:
            *   LBドメインの複数のレコードにおいて、LBDTC（検査実施日）と、SVドメインに記録されているVISITDY（計画された訪問日）にずれが発生しています。
            *   LBドメインのLBDTCは、いつ検査を実施したものでしょうか？VISITDYと一致しない理由があれば教えてください。
        *   判断理由: 検査日が計画された訪問日から大きくずれている場合、臨床試験の評価に影響を与える可能性があるため。

    *   クエリNo.: 4
        *   臨床試験結果への影響度合い: Minor
        *   変数名と値: DM.SEX = "M", MH.MHTERM = "ALZHEIMER'S DISEASE"
        *   医療機関への問い合わせ文面:
            *   DM.SEX（性別）が「M（男性）」である被験者について、MH.MHTERM（既往歴）に「ALZHEIMER'S DISEASE（アルツハイマー病）」の記録がありますが、医学的に妥当でしょうか。また、本症例はTIドメインのInclusion Criteriaを満たしているか確認してください。
        *   判断理由: 
            *   Inclusion Criteriaとの整合性を確認する必要があるため。
            *   DM.SEXとMH.MHTERMの組み合わせが医学的に矛盾しないか確認する必要があるため。
```

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.043462
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.199987
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.238825
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T: 0.5) gemini-2.0-flash-exp 
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T: 0.3) gemini-2.0-flash-exp
------
Event: node_finished
Node: (1 T: 0.7) gemini-2.0-flas

...

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.041261
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.292104
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.246711
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T: 0.5) gemini-2.0-flash-exp 
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T: 0.3) gemini-2.0-flash-exp
------
Event: node_finished
Node: (4 T: 0.5) gemini-2.0-flas

**データ整合性レビュー結果**

全体的なデータ品質の評価:

*   総合評価: 一部問題あり
*   データクリーニング/再調査が必要な項目: CM.CMENDTC, CM.CMDOSU, MH.MHSTDTC, AE.AESTDTC, AE.AEENDTC, VS.VSPOS, QS.QSORRES, SE.SEENDTC, LB.LBORRESU, LB.LBSTRESU, EX.EXSTDTC

問題点:

*   問題No.: 1
    *   変数名と値: CM.CMENDTC = "" (空文字), CM.CMSTDTC = "2008" (年のみ), CM.USUBJID = "01-701-1146", CM.CMTRT = "BECONASE, DOCUSATE, MINERALS NOS, MOTRIN, MULTIVITAMIN, PREMARIN, PROVERA"
    *   矛盾の内容: CM.CMENDTCが空文字であるにも関わらず、CM.CMSTDTCが年のみで日付が不完全。併用薬の使用期間が不明確。CMENDTCが空欄の場合、CMTRTの投与が継続していることを意味すると思われるが、CMSTDTCに年のみの値が入っているため、矛盾している。
    *   問題点の原因（推測）: データ入力エラー、または併用薬の使用期間が不明、CMENDTCのデータ収集漏れ。
    *   対応策（提案）: 
        1.  CM.CMENDTCに適切な終了日を入力するか、併用薬が継続中の場合はその旨を示す値を入力する。
        2.  CM.CMSTDTCに可能な限り正確な開始日を入力する。
        3.  CM.CMENDTCとCM.CMSTDTCが両方とも欠損している場合は、CMドメインへの記録自体が適切かどうかを検討する。
        4.  CMENDTCにデータが入っていない理由を確認。

*   問題No.: 2
    *   変数名と値: CM.CMDOSU = "ug", CM.CMDOSE = 2.5, CM.CMTRT = "PROVERA", CM.USUBJID = "01-701-1146"
    *   矛盾の内容: CM.CMDOSU (Dose Units) が "ug" (マイクログラム) であるにもかかわらず、CM.CMDOSE (Dose per Administration) が 2.5 と非常に小さい値。PROVERAのCMDOSU（Dose Units）が"ug"（マイクログラム）となっていますが、CMDOSE（Dose per Administration）が2.5であり、PROVERAの通常投与量として2.5ugは少なすぎる。
    *   問題点の原因（推測）: データ入力エラー、単位の誤り。
    *   対応策（提案）: 
        1.  CMDOSUとCMDOSEの値が正しいか確認する。単位が誤っている場合は修正する。
        2.  CMDOSUが誤っている場合、正しい単位（例：mg）と投与量を確認してください。Define.xmlのCMDOSUのCodeListにmgを追加することも検討してください。

*   問題No.: 3
    *   変数名と値: MH.MHSTDTC = "", MH.USUBJID = "01-701-1146", MH.MHTERM = "ALZHEIMER'S DISEASE, POLYARTHRITIS, BLADDER INFECTION, HEARING LOSS, CONSTIPATION, CHEST PAIN, BLADDER PROLAPSE, ESOPHAGEAL SPASM, DIARRHEA, HEART MURMUR, DEPRESSION, KNEE PAIN, SINUSITIS"
    *   矛盾の内容: 既往歴の開始日が空欄。MHSTDTCが空欄である。
    *   問題点の原因（推測）: データ入力エラー、または正確な開始日が不明、MHSTDTCのデータ収集漏れ。
    *   対応策（提案）: 
        1.  MH.MHSTDTCに可能な限り正確な開始日を入力する。
        2.  開始日が不明な場合は、その旨を示す値を入力する。

*   問題No.: 4
    *   変数名と値: AE.AESTDTC = "2013-06-10", AE.AEENDTC = "", AE.USUBJID = "01-701-1146", AE.AETERM = "APPLICATION SITE ERYTHEMA, APPLICATION SITE IRRITATION, APPLICATION SITE PAIN, APPLICATION SITE PRURITUS, APPLICATION SITE VESICLES, FATIGUE, RASH"
    *   矛盾の内容: 有害事象の開始日は存在するが、終了日が空欄。
    *   問題点の原因（推測）: データ入力エラー、または有害事象が継続中。
    *   対応策（提案）:
        1.  AE.AEENDTCに適切な終了日を入力するか、有害事象が継続中の場合はその旨を示す値を入力する。

*   問題No.: 5
    *   変数名と値: VS.VSPOS = "", VS.VSTEST = "Height", VS.USUBJID = "01-701-1146"
    *   矛盾の内容: USUBJIDが"01-701-1146"の被験者の身長測定時の体位(VSPOS)が空欄になっている。
    *   問題点の原因（推測）: VSPOSのデータ入力漏れ。
    *   対応策（提案）: VSPOSに適切な体位(STANDING, SUPINEなど)を入力する。

*   問題No.: 6
    *   変数名と値: QS.QSORRES = "Y/NA", QS.QSTEST = "UNDERTAKE TO GO OUT, SAFELY TAKE CAR, BUS, TAXI, DECIDE TO TAKE HIS/HER MEDICATIONS", QS.USUBJID = "01-701-1146"
    *   矛盾の内容: DAD評価項目のQSORRESが"Y"または"NA"となっている。DAD評価項目のQSORRESは、数値で評価される項目であるため、"Y"または"NA"は不適切である。
    *   問題点の原因（推測）: QSORRESのデータ入力誤り。
    *   対応策（提案）: QSORRESに適切な数値を入力する。

*   問題No.: 7
    *   変数名と値: SE.SEENDTC = "2013-06-30", DS.DSSTDTC = "2013-06-30", DS.DSTERM = "ADVERSE EVENT", SE.ELEMENT = "High_Middle", SE.USUBJID = "01-701-1146"
    *   矛盾の内容: USUBJIDが"01-701-1146"の被験者のHigh_Middle (HIM)のSEENDTCが2013-06-30となっているが、DS.DSTERMが"ADVERSE EVENT"となっている。有害事象により、治験薬の投与が中断された場合、SEENDTCはDS.DSSTDTCと一致する必要がある。
    *   問題点の原因（推測）: SEENDTCのデータ入力誤り。
    *   対応策（提案）: SEENDTCにDS.DSSTDTCの値を入力する。

*   問題No.: 8
    *   変数名と値: AE.AESTDY = -4, AE.AEENDY = 14, AE.AESTDTC = "2013-05-16", AE.AEENDTC = "2013-06-02", AE.AETERM = "RASH", AE.USUBJID = "01-701-1146"
    *   矛盾の内容: USUBJIDが"01-701-1146"の被験者の有害事象である"RASH"のAESTDYが-4、AEENDYが14となっている。AESTDYはAESTDTCから算出されるはずだが、AESTDTCが2013-05-16であり、RFSTDTCが2013-05-20であるため、AESTDYは-4となるのは正しい。しかし、AEENDTCが2013-06-02であり、RFSTDTCが2013-05-20であるため、AEENDYは13となるはずである。
    *   問題点の原因（推測）: AEENDYの算出誤り。
    *   対応策（提案）: AEENDYの値を13に修正する。

**クエリ**

*   患者ID: 01-701-1146
    *   クエリNo.: 1
        *   臨床試験結果への影響度合い: Major
        *   変数名と値: CM.CMENDTC, CM.CMSTDTC (BECONASE, DOCUSATE, MINERALS NOS, MOTRIN, MULTIVITAMIN, PREMARIN, PROVERA, PAROXETINE)
        *   医療機関への問い合わせ文面:
            *   患者[01-701-1146]の併用薬について、BECONASE, DOCUSATE, MINERALS NOS, MOTRIN, MULTIVITAMIN, PREMARIN, PROVERA, PAROXETINEのそれぞれの正確な使用期間（開始日と終了日）をご教示ください。終了日が不明な場合は、現在も使用中かどうかをお知らせください。
        *   判断理由: 併用薬の使用期間は、有害事象の評価や試験結果の解釈に重要な情報であり、データ品質に大きく影響するため。

    *   クエリNo.: 2
        *   臨床試験結果への影響度合い: Major
        *   変数名と値: CM.CMDOSU (PROVERA)
        *   医療機関への問い合わせ文面:
            *   患者[01-701-1146]のPROVERAの投与量単位が「ug」となっていますが、mgの間違いではないでしょうか？正しい投与量と単位をご教示ください。
        *   判断理由: PROVERAの用量単位がugとmgで大きく異なり、臨床的に投与量単位が誤っている可能性が高い。

    *   クエリNo.: 3
        *   臨床試験結果への影響度合い: Minor
        *   変数名と値: MH.MHSTDTC (ALZHEIMER'S DISEASE, POLYARTHRITIS, BLADDER INFECTION, HEARING LOSS, CONSTIPATION, CHEST PAIN, BLADDER PROLAPSE, ESOPHAGEAL SPASM, DIARRHEA, HEART MURMUR, DEPRESSION, KNEE PAIN, SINUSITIS)
        *   医療機関への問い合わせ文面:
            *   患者[01-701-1146]の既往歴について、ALZHEIMER'S DISEASE, POLYARTHRITIS, BLADDER INFECTION, HEARING LOSS, CONSTIPATION, CHEST PAIN, BLADDER PROLAPSE, ESOPHAGEAL SPASM, DIARRHEA, HEART MURMUR, DEPRESSION, KNEE PAIN, SINUSITISそれぞれの開始日をご教示ください。
        *   判断理由: 既往歴の開始日は、有害事象との関連性を評価する上で重要な情報であるため。

    *   クエリNo.: 4
        *   臨床試験結果への影響度合い: Minor
        *   変数名と値: AE.AEENDTC (APPLICATION SITE ERYTHEMA, APPLICATION SITE IRRITATION, APPLICATION SITE PAIN, APPLICATION SITE PRURITUS, APPLICATION SITE VESICLES, FATIGUE, RASH)
        *   医療機関への問い合わせ文面:
            *   患者[01-701-1146]の有害事象について、APPLICATION SITE ERYTHEMA, APPLICATION SITE IRRITATION, APPLICATION SITE PAIN, APPLICATION SITE PRURITUS, APPLICATION SITE VESICLES, FATIGUE, RASHそれぞれの転帰日をご教示ください。
        *   判断理由: 有害事象の転帰日は、安全性評価において重要な情報であるため。

    *   クエリNo.: 5
        *   臨床試験結果への影響度合い: Major
        *   変数名と値: SE.SEENDTC = "2013-06-30", DS.DSSTDTC = "2013-06-30", DS.DSTERM = "ADVERSE EVENT", SE.ELEMENT = "High_Middle"
        *   医療機関への問い合わせ文面:
            *   患者[01-701-1146]のHigh_Middle (HIM)のSEENDTCが2013-06-30となっており、DS.DSTERMが"ADVERSE EVENT"となっています。有害事象により、治験薬の投与が中断された場合、SEENDTCはDS.DSSTDTCと一致する必要があります。SEENDTCの値が正しいか確認をお願いします。
        *   判断理由: 治験薬の投与期間は、試験結果に影響を与える可能性があるため、正確な情報を把握する必要があります。
```

--- 試行回数: 1 ---
Event: workflow_started
------
Event: node_started
Node: 開始
------
Event: node_finished
Node: 開始 (succeeded)
Error: None
Elapsed time: 0.046901
------
Event: node_started
Node: テキスト抽出ツール
------
Event: node_finished
Node: テキスト抽出ツール (succeeded)
Error: None
Elapsed time: 0.252351
------
Event: node_started
Node: IF/ELSE
------
Event: node_finished
Node: IF/ELSE (succeeded)
Error: None
Elapsed time: 0.132341
------
Event: parallel_branch_started
------
Event: node_started
Node: (1 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (2 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (3 T: 0.7) gemini-2.0-flash-exp
------
Event: parallel_branch_started
------
Event: node_started
Node: (4 T: 0.5) gemini-2.0-flash-exp 
------
Event: parallel_branch_started
------
Event: node_started
Node: (5 T: 0.3) gemini-2.0-flash-exp
------
Event: node_finished
Node: (3 T: 0.7) gemini-2.0-flas

**患者ID: 01-701-1146**

*   **逸脱No.: 1**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: CM.CMDOSU = "ug"、CM.CMDOSE = 2.5、CM.CMTRT = "PROVERA"
    *   逸脱内容: 併用薬PROVERAの投与量の単位がugで記録されているが、通常mgが使用されるべき。
    *   プロトコル該当箇所: 併用薬に関する規定（具体的なセクション番号は不明）
    *   判断理由: PROVERA（メドロキシプロゲステロン酢酸エステル）は通常、2.5mg等の用量で使用される薬剤であり、ug（マイクログラム）での投与は一般的ではないため、単位の誤りの可能性がある。データ入力の誤りである可能性も考慮される。

*   **逸脱No.: 2**
    *   臨床試験結果への影響度合い: Critical
    *   変数名と値: MH.MHTERM = "ALZHEIMER'S DISEASE"
    *   逸脱内容: 既往歴にアルツハイマー病の記載がある。
    *   プロトコル該当箇所: 3.4.2.2 Exclusion Criteria
    *   判断理由: TI.IETEST = "EXCL12", TI.IETEST = "Diagnosis of serious neurological conditions.", MH.MHTERM = "ALZHEIMER'S DISEASE", 既往歴にアルツハイマー病（AD）の記載があるため、除外基準EXCL12に抵触する。ただし、DM.TDIGRP="Patients with Probable Mild to Moderate Alzheimer's Disease"より、治験対象患者はアルツハイマー病患者であるため、データ入力の誤りである可能性がある。

*   **逸脱No.: 3**
    *   臨床試験結果への影響度合い: Major
    *   変数名と値: AE.AESTDTC = "2013-06-10",AE.AEDTC = "2013-06-16" 、AE.AEENDTC = ""
    *   逸脱内容: AEのAESTDTCとAEDTCの矛盾。
    *   プロトコル該当箇所:3.9.3.2.1. Adverse Event Reporting Requirements
    *   判断理由: AE5,7,6,8において、有害事象の開始日（AESTDTC）は2013-06-10ですが、データ収集日（AEDTC）は2013-06-16となっており、矛盾しています。データ収集日よりも前に有害事象が発生している必要があるので、AESTDTCに誤りがあると考えられます。また、AEENDTCがNullであることも不自然です。

*   **逸脱No.: 4**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 54、EX.EXSTDTC = "2013-05-20"、EX.EXENDTC = "2013-06-03"
    *   逸脱内容: 治験薬XANOMELINE54mgの投与期間が15日間となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design
    *   判断理由: 試験デザインでは、投与期間は2週間ごと、または4週間ごとである。投与期間が15日間となっているのは、投与計画からの逸脱と考えられる。

*   **逸脱No.: 5**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 81、EX.EXSTDTC = "2013-06-04"、EX.EXENDTC = "2013-06-26"
    *   逸脱内容: 治験薬XANOMELINE81mgの投与期間が23日間となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design
    *   判断理由: 試験デザインでは、投与期間は2週間ごと、または4週間ごとである。投与期間が23日間となっているのは、投与計画からの逸脱と考えられる。

*   **逸脱No.: 6**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: DM.AGE = 75、MH.MHTERM = "TONSILLECTOMY"、MH.MHSTDTC = "1942"
    *   逸脱内容: 75歳の被験者の既往歴に、1942年に扁桃摘出術が記録されている。
    *   プロトコル該当箇所: なし
    *   判断理由: 75歳という年齢を考慮すると、1942年に扁桃摘出術を受けたとするには、患者の年齢と手術年との間に矛盾が生じる可能性があり、データに誤りがないか確認する必要がある。

*   **逸脱No.: 7**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM25", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM25(UNDERTAKE TO GO OUT)の回答が、BASELINE時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 8**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM26", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM26(ADEQUATELY ORGANIZE AN OUTING)の回答が、BASELINE時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 9**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM28", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM28(SAFELY TAKE CAR, BUS, TAXI)の回答が、BASELINE時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 10**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM29", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM29(RETURN FROM THE STORE)の回答が、BASELINE時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 11**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM34", QSORRES = "NA", VISIT = "WEEK 6"
    *   逸脱内容: DADの質問項目DAITM34(DECIDE TO TAKE HIS/HER MEDICATIONS)の回答が、WEEK6時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、WEEK6時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 12**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM40", QSORRES = "NA", VISIT = "WEEK 6"
    *   逸脱内容: DADの質問項目DAITM40(STAY SAFELY AT HOME BY HIMSELF/HERSELF)の回答が、WEEK6時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、WEEK6時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **逸脱No.: 13**
    *   臨床試験結果への影響度合い: Major
    *   変数名と値: VS.VSTESTCD = "DIABP", VS.VISIT = "WEEK 2", VS.VSSTRESN = 43
    *   逸脱内容: 仰臥位から立位への体位変換後の血圧低下
    *   プロトコル該当箇所:  3.9.3.4.1 Vital Sign Determination
    *   判断理由: WEEK2において、仰臥位でのDIABPが57mmHgに対し、立位1分後には43mmHgまで低下している。起立性低血圧の疑いがあり、安全性に影響する可能性がある。

*   **逸脱No.: 14**
    *   臨床試験結果への影響度合い: None
    *   変数名と値: AE.AETERM = "RASH"、AE.AEREL = "NONE"
    *   逸脱内容: 治験薬との因果関係が否定されたRASHが発現している。
    *   プロトコル該当箇所: 3.9.3.2.1 Adverse Event Reporting Requirements.
    *   判断理由: 治験薬との因果関係が否定されているため、プロトコル逸脱には該当しない。

*   **逸脱No.: 15**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: DM.RFXENDTC = "2013-06-26", DS.DSSTDTC = "2013-06-30"
    *   逸脱内容: 最終投与日よりも後のFinal Visit
    *   プロトコル該当箇所: 3.1 Summary of Study Design
    *   判断理由: 
治験薬の最終投与日は2013-06-26であるにも関わらず、DS.DSDECODがFINAL LAB VISITであるDSのDSSTDTCが2013-06-30となっているため、矛盾している。

*   **逸脱No.: 16**
    *   臨床試験結果への影響度合い: None
    *   変数名と値: NPITM04～NPITM12, VISIT=WEEK4
    *   逸脱内容: Neuropsychiatric Inventory (NPI-X) のデータがABSENTにも関わらず、スコアが存在する。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: NPI-Xは、介護者が患者の行動を評価するものであり，行動がABSENTの場合は、スコアを0とすることになっている。データに矛盾はない。

*   **逸脱No.: 17**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: VS.VISIT = "AMBUL ECG PLACEMENT", VS.VISITDY = 13
    *   逸脱内容: AMBUL ECG PLACEMENTのVISITDYが13日目となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design, Protocol Attachment LZZT.1
    *   判断理由: AMBUL ECG PLACEMENTはSCREENING2とBASELINEの間に行われるべきであり、WEEK2(14日目)と矛盾している。

*   **逸脱No.: 18**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: VS.VISIT = "AMBUL ECG REMOVAL", VS.VISITDY = 30
    *   逸脱内容: AMBUL ECG REMOVALのVISITDYが30日目となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design, Protocol Attachment LZZT.1
    *   判断理由: AMBUL ECG REMOVALはWEEK 2(14日目)とWEEK 4(28日目)の間に行われるべきであり、WEEK6(42日目)と矛盾している。

*   **逸脱No.: 19**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: AE.AESEQ = 1, AE.AESTDY = -4
    *   逸脱内容: AESTDYが負の値になっている。
    *   プロトコル該当箇所: AE
    *   判断理由: AESTDYが負の値になっている。

*   **逸脱No.: 20**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXSTDY = 1, EX.EXENDY = 15, SE.SESTDTC = "2013-05-20", SE.SEENDTC = "2013-06-03", SE.ETCD = "HIS"
    *   逸脱内容: EXとSEのデータの不一致
    *   プロトコル該当箇所: EX, SE
    *   判断理由: EXのEXSTDYとEXENDYの値から計算すると、治験期間は15日となる。しかしSEのSESTDTCとSEENDTCの値から計算すると、治験期間は14日となる。データに不一致がある。

*   **逸脱No.: 21**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXSTDY = 16, EX.EXENDY = 38, SE.SESTDTC = "2013-06-03", SE.SEENDTC = "2013-06-30", SE.ETCD = "HIM"
    *   逸脱内容: EXとSEのデータの不一致
    *   プロトコル該当箇所: EX, SE
    *   判断理由: EXのEXSTDYとEXENDYの値から計算すると、治験期間は22日となる。しかしSEのSESTDTCとSEENDTCの値から計算すると、治験期間は27日となる。データに不一致がある。

*   **逸脱No.: 22**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: DM.DMDY = -13
    *   逸脱内容: DMDYが負の値になっている。
    *   プロトコル該当箇所: DM
    *   判断理由: DMDYが負の値になっている。

*   **逸脱No.: 23**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: LB.LBDY = -13
    *   逸脱内容: LBDYが負の値になっている。
    *   プロトコル該当箇所: LB
    *   判断理由: LBのLBDYが負の値になっている。

*   **逸脱No.: 24**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: SC.SCDY = -13
    *   逸脱内容: SCDYが負の値になっている。
    *   プロトコル該当箇所: SC
    *   判断理由: SCのSCDYが負の値になっている。

*   **逸脱No.: 25**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSDY = -13
    *   逸脱内容: QSDYが負の値になっている。
    *   プロトコル該当箇所: QS
    *   判断理由: QSのQSDYが負の値になっている。

*   **逸脱No.: 26**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: SV.VISITDY = null
    *   逸脱内容: VISITDYがNULLになっている。
    *   プロトコル該当箇所: SV
    *   判断理由: SVのVISITDYがNULLになっている。

*   **クエリNo.: 1**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: CM.CMDOSU = "ug"、CM.CMDOSE = 2.5、CM.CMTRT = "PROVERA"
    *   医療機関への問い合わせ文面: 併用薬PROVERAの投与量の単位について、正しくはmgではないでしょうか？ugで投与された場合、その理由をご教示ください。
    *   判断理由: PROVERAの投与単位がugとなっていることが医学的に不自然であるため、確認が必要。

*   **クエリNo.: 2**
    *   臨床試験結果への影響度合い: Critical
    *   変数名と値: MH.MHTERM = "ALZHEIMER'S DISEASE"
    *   医療機関への問い合わせ文面: 本患者はアルツハイマー病患者として登録されていますが、既往歴にもアルツハイマー病の記載があります。これはデータ入力の誤りでしょうか、それともプロトコル違反に該当するのでしょうか？
    *   判断理由: アルツハイマー病患者を対象とした臨床試験において、既往歴にアルツハイマー病の記載があることは、選択基準または除外基準に抵触する可能性があるため、確認が必要。

*   **クエリNo.: 3**
    *   臨床試験結果への影響度合い: Major
    *   変数名と値: AE.AESTDTC = "2013-06-10",AE.AEDTC = "2013-06-16"
    *   医療機関への問い合わせ文面: 有害事象「APPLICATION SITE ERYTHEMA」の発現日（AESTDTC）とデータ収集日（AEDTC）に矛盾が見られます。発現日の記録に誤りがないか確認してください。また、有害事象のAEOUT, AEENDTCがNullになっている理由を教えてください。
    *   判断理由: データ収集日よりも前に有害事象が発生している必要があるので、AESTDTCに誤りがあると考えられます。

*   **クエリNo.: 4**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 54、EX.EXSTDTC = "2013-05-20"、EX.EXENDTC = "2013-06-03"
    *   医療機関への問い合わせ文面: 治験薬XANOMELINE54mgの投与期間が15日間となっていますが、投与計画どおり2週間（14日間）の投与で間違いないでしょうか？
    *   判断理由: 投与期間がプロトコルで定められた期間と異なっている可能性があるため、確認が必要。

*   **クエリNo.: 5**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 81、EX.EXSTDTC = "2013-06-04"、EX.EXENDTC = "2013-06-26"
    *   医療機関への問い合わせ文面: 治験薬XANOMELINE81mgの投与期間が23日間となっていますが、投与計画どおり4週間（28日間）の投与で間違いないでしょうか？
    *   判断理由: 投与期間がプロトコルで定められた期間と異なっている可能性があるため、確認が必要。

*   **クエリNo.: 6**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: MH.MHDTC = "2013-05-07", MH.MHSTDTC = "1942"、DM.AGE = 75
    *   医療機関への問い合わせ文面: 患者は75歳であるのに対し、既往歴(MH)に"TONSILLECTOMY"が1942年に実施されたと記録されています。75歳という年齢を考慮すると、1942年に扁桃摘出術を受けたとするには、患者の年齢と手術年との間に矛盾が生じる可能性があり、データに誤りがないか確認をお願いします。
    *   判断理由: 年齢と既往歴の整合性の確認が必要であるため。

*   **クエリNo.: 7**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM25", QSORRES = "NA", VISIT = "BASELINE"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM25(UNDERTAKE TO GO OUT)の回答が、BASELINE時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 8**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM26", QSORRES = "NA", VISIT = "BASELINE"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM26(ADEQUATELY ORGANIZE AN OUTING)の回答が、BASELINE時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 9**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM28", QSORRES = "NA", VISIT = "BASELINE"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM28(SAFELY TAKE CAR, BUS, TAXI)の回答が、BASELINE時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 10**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM29", QSORRES = "NA", VISIT = "BASELINE"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM29(RETURN FROM THE STORE)の回答が、BASELINE時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 11**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM34", QSORRES = "NA", VISIT = "WEEK 6"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM34(DECIDE TO TAKE HIS/HER MEDICATIONS)の回答が、WEEK6時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、WEEK6時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 12**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM40", QSORRES = "NA", VISIT = "WEEK 6"
    *   医療機関への問い合わせ文面: DADの質問項目DAITM40(STAY SAFELY AT HOME BY HIMSELF/HERSELF)の回答が、WEEK6時に"NA"となっています。評価がNAとなった理由を教えてください。
    *   判断理由: DADは活動能力を評価するものであり、WEEK6時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   **クエリNo.: 13**
    *   臨床試験結果への影響度合い: Major
    *   変数名と値: VS.VSTESTCD = "DIABP", VS.VISIT = "WEEK 2", VS.VSSTRESN = 43
    *   医療機関への問い合わせ文面: WEEK 2のバイタルサイン測定において、仰臥位から立位への体位変換後にDIABPが大きく低下していますが、起立性低血圧の症状や既往歴について確認してください。また、患者の状態について詳細をご教示ください。
    *   判断理由: 患者の安全に関わる可能性があるため、医療機関への確認が必要。

*   **クエリNo.: 14**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: AE.AETERM = "RASH"、AE.AEREL = "NONE"
    *   医療機関への問い合わせ文面: RASHの原因について、治験薬以外に考えられる要因はありますか？
    *   判断理由: 治験薬との因果関係が否定されているRASHについて、他の要因の可能性を確認するため。

*   **クエリNo.: 15**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXSTDY = 1, EX.EXENDY = 15, SE.SESTDTC = "2013-05-20", SE.SEENDTC = "2013-06-03", SE.ETCD = "HIS"
    *   医療機関への問い合わせ文面: EXとSEのデータの不一致について確認します。EXのEXSTDYとEXENDYの値から計算すると、治験期間は15日となる。しかしSEのSESTDTCとSEENDTCの値から計算すると、治験期間は14日となる。どちらが正しいでしょうか？
    *   判断理由: データに不一致があるため。

*   **クエリNo.: 16**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXSTDY = 16, EX.EXENDY = 38, SE.SESTDTC = "2013-06-03", SE.SEENDTC = "2013-06-30", SE.ETCD = "HIM"
    *   医療機関への問い合わせ文面: EXとSEのデータの不一致について確認します。EXのEXSTDYとEXENDYの値から計算すると、治験期間は22日となる。しかしSEのSESTDTCとSEENDTCの値から計算すると、治験期間は27日となる。どちらが正しいでしょうか？
    *   判断理由: データに不一致があるため。

*   **クエリNo.: 17**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: VS.VISIT = "AMBUL ECG PLACEMENT", VS.VISITDY = 13
    *   医療機関への問い合わせ文面: AMBUL ECG PLACEMENTのVISITDYが13日目となっています。正しい日付を教えて下さい。
    *   判断理由: データに矛盾があるため。

*   **クエリNo.: 18**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: VS.VISIT = "AMBUL ECG REMOVAL", VS.VISITDY = 30
    *   医療機関への問い合わせ文面: AMBUL ECG REMOVALのVISITDYが30日目となっています。正しい日付を教えて下さい。
    *   判断理由: データに矛盾があるため。

*   **クエリNo.: 19**
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: AE.AESTDY = -4
    *   医療機関への問い合わせ文面: AE.AESTDYが負の値になっています。発現日を確認して正しい値を教えてください。
    *   判断理由: AESTDYが負の値になっているため

**ASSISTANT6:**
患者ID: 01-701-1146
*   逸脱No.: 1
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: CM.CMDOSU = "ug"、CM.CMDOSE = 2.5、CM.CMTRT = "PROVERA"
    *   逸脱内容: 併用薬PROVERAの投与量の単位がugで記録されているが、通常mgが使用されるべき。
    *   プロトコル該当箇所: 併用薬に関する規定（具体的なセクション番号は不明）
    *   判断理由: PROVERA（メドロキシプロゲステロン酢酸エステル）は通常、2.5mg等の用量で使用される薬剤であり、ug（マイクログラム）での投与は一般的ではないため、単位の誤りの可能性がある。

*   逸脱No.: 2
    *   臨床試験結果への影響度合い: Critical
    *   変数名と値: MH.MHTERM = "ALZHEIMER'S DISEASE"
    *   逸脱内容: 既往歴にアルツハイマー病の記載がある。
    *   プロトコル該当箇所: 3.4.2.2 Exclusion Criteria
    *   判断理由: TI.IETEST = "EXCL12", TI.IETEST = "Diagnosis of serious neurological conditions.", MH.MHTERM = "ALZHEIMER'S DISEASE", 既往歴にアルツハイマー病（AD）の記載があるため、除外基準EXCL12に抵触する。ただし、DM.TDIGRP="Patients with Probable Mild to Moderate Alzheimer's Disease"より、治験対象患者はアルツハイマー病患者であるため、データ入力の誤りである可能性がある。

*   逸脱No.: 3
    *   臨床試験結果への影響度合い: Major
    *   変数名と値: AE.AESTDTC = "2013-06-10",AE.AEDTC = "2013-06-16" 、AE.AEENDTC = ""
    *   逸脱内容: AEのAESTDTCとAEDTCの矛盾。
    *   プロトコル該当箇所:3.9.3.2.1. Adverse Event Reporting Requirements
    *   判断理由: AE5,7,6,8において、有害事象の開始日（AESTDTC）は2013-06-10ですが、データ収集日（AEDTC）は2013-06-16となっており、矛盾しています。データ収集日よりも前に有害事象が発生している必要があるので、AESTDTCに誤りがあると考えられます。また、AEENDTCがNullであることも不自然です。

*   逸脱No.: 4
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 54、EX.EXSTDTC = "2013-05-20"、EX.EXENDTC = "2013-06-03"
    *   逸脱内容: 治験薬XANOMELINE54mgの投与期間が15日間となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design
    *   判断理由: 試験デザインでは、投与期間は2週間ごと、または4週間ごとである。投与期間が15日間となっているのは、投与計画からの逸脱と考えられる。

*   逸脱No.: 5
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: EX.EXTRT = "XANOMELINE"、EX.EXDOSE = 81、EX.EXSTDTC = "2013-06-04"、EX.EXENDTC = "2013-06-26"
    *   逸脱内容: 治験薬XANOMELINE81mgの投与期間が23日間となっている。
    *   プロトコル該当箇所: 3.1 Summary of Study Design
    *   判断理由: 試験デザインでは、投与期間は2週間ごと、または4週間ごとである。投与期間が23日間となっているのは、投与計画からの逸脱と考えられる。

*   逸脱No.: 6
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: DM.AGE = 75
    *   逸脱内容: 年齢とMedical Historyの整合性
    *   プロトコル該当箇所:  なし
    *   判断理由: 患者は75歳であるのに対し、既往歴(MH)に"TONSILLECTOMY"が1942年に実施されたと記録されている。75歳という年齢を考慮すると、1942年に扁桃摘出術を受けたとするには、患者の年齢と手術年との間に矛盾が生じる可能性があり、データに誤りがないか確認する必要がある。

*   逸脱No.: 7
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM25", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM25(UNDERTAKE TO GO OUT)の回答が、BASELINE時に"NA"となっている。
    *   プロトコル該当箇所: 3.9.1.1 Efficacy Measures
    *   判断理由: DADは活動能力を評価するものであり、BASELINE時に「該当しない」という回答は通常あり得ない。評価の妥当性を確認する必要があるため。

*   逸脱No.: 8
    *   臨床試験結果への影響度合い: Minor
    *   変数名と値: QS.QSTESTCD = "DAITM26", QSORRES = "NA", VISIT = "BASELINE"
    *   逸脱内容: DADの質問項目DAITM26(ADEQUATELY ORGANIZE AN OUTING)の回答が、BASELINE時に"NA"となっている。


,Subject,Task1,Task2,Task3
0,01-701-1097,## 症例サマリー\n\n**患者ID:** 01-701-1097\n\n* 2013...,**全体的なデータ品質の評価:**\n* 総合評価: 一部問題あり\n* データクリ...,**患者ID:** 01-701-1097\n\n* **逸脱No.:** 1\n ...
1,01-701-1387,**症例サマリー**\n\n**患者ID:** 01-701-1387\n\n* 201...,### 全体的なデータ品質の評価:\n\n* 総合評価: 一部問題あり\n* データ...,**患者ID:** 01-701-1387\n\n* **逸脱No.:** 1\n ...
2,01-701-1028,## 症例サマリー\n\n* **患者ID:** 01-701-1028\n * ...,```markdown\n全体的なデータ品質の評価:\n * 総合評価: 一部問題あり...,## プロトコル逸脱の検出\n\n* **患者ID:** 01-701-1028\n ...
3,01-704-1010,## 1. 症例サマリーの統合\n\n**患者ID:** 01-704-1010\n\n* ...,### データ品質レビュー結果\n\n**全体的なデータ品質の評価:**\n\n* 総合...,## プロトコル逸脱の検出\n\n* 患者ID: 01-704-1010\n * ...
4,01-701-1148,### 症例サマリー\n\n**患者ID:** 01-701-1148\n\n* 201...,### 全体的なデータ品質の評価:\n\n* 総合評価: 一部問題あり\n* データ...,## プロトコル逸脱の検出\n\n* 患者ID: 01-701-1148\n * ...
5,01-701-1023,## 症例サマリー\n\n**患者ID:** 01-701-1023\n\n* 2012...,**全体的なデータ品質の評価:**\n\n* 総合評価: 一部問題あり\n* データ...,## プロトコル逸脱の検出\n\n* **患者ID:** 01-701-1023\n ...
6,01-703-1335,以下に5人のアシスタントの回答を統合した結果を示します。\n\n**1. 症例サマリー:**...,はい、承知いたしました。5人のアシスタントの回答を統合し、指定されたフォーマットに従って出力...,**患者ID:** 01-703-1335\n\n* **逸脱No.:** 1\n ...
7,01-703-1403,## 症例サマリー (統合)\n\n**患者ID:** 01-703-1403\n\n* ...,### 全体的なデータ品質の評価:\n\n* 総合評価: 一部問題あり\n* データ...,**統合結果:**\n\n患者ID: 01-703-1403\n\n* **逸脱No.:...
8,01-701-1181,**患者ID:** 01-701-1181\n\n* **2013年11月26日 (Da...,**全体的なデータ品質の評価:**\n\n* 総合評価: 一部問題あり\n* データ...,**患者ID:** 01-701-1181\n\n* **逸脱No.:** 1\n ...
9,01-701-1015,## 症例サマリー\n\n**患者ID:** 01-701-1015\n\n* 2013...,### 全体的なデータ品質の評価:\n\n* 総合評価: 一部問題あり、要修正\n* ...,以下に、5人のアシスタントの回答を統合し、指定のフォーマットに従って出力します。\n\n**...


In [25]:
def dataframe_to_text(df: pd.DataFrame) -> str:
    """
    DataFrameを指定されたテキスト形式に変換します。

    Args:
        df: 変換するDataFrame。カラム名は 'Subject', 'Task1', 'Task2', 'Task3' である必要があります。

    Returns:
        変換後のテキストデータ。
    """
    text_data = ""
    for index, row in df.iterrows():
        subject = row['Subject']
        task1 = row['Task1']
        task2 = row['Task2']
        task3 = row['Task3']

        text_data += f"# {subject}\n"
        text_data += f"## Task1: Clinical Review Results\n"
        text_data += f"{task1}\n"
        text_data += f"## Task2: DM Review Results\n"
        text_data += f"{task2}\n"
        text_data += f"## Task3: Protocol Deviation Review Results\n"
        text_data += f"{task3}\n\n"

    return text_data
output_text = dataframe_to_text(df_results)

In [26]:
# mdファイルに保存
output_file = 'output_' + ModelName + '.md'  # 保存するファイル名を指定
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(output_text)

print(output_text)

# 01-701-1097
## Task1: Clinical Review Results
## 症例サマリー

**患者ID:** 01-701-1097

*   2013年12月23日 (Day -9): スクリーニング検査で、ナトリウムが基準値より低い(134 mEq/L、基準範囲: 135-145 mEq/L)。
*   2014年02月20日 (Day 51): 適用部位小胞を発症。重症度は軽度で、2014年2月22日に回復。
*   2014年02月20日 (Day 51): 全身性掻痒症を発症。重症度は中等度で、2014年3月21日に回復。
*   2014年02月21日 (Day 52): 適用部位そう痒症が発生。重症度は軽度。
*   2014年02月26日 (Day 57): Week8の検査で、好酸球数が基準値より高い(0.89 THOU/uL, 基準値: 0-0.57 THOU/uL)。
*   2014年03月26日 (Day 85): Week12の検査で、好酸球数が基準値より高い(0.87 THOU/uL, 基準値: 0-0.57 THOU/uL)。
*   2014年04月19日 (Day 109): 鼻詰まりと咽喉頭痛を発症。重症度はいずれも軽度で、2014年4月22日に回復。
*   2014年05月21日 (Day 141): 収縮期血圧が低下(112 mmHg、臥位)。
*   2014年05月21日 (Day 141): 好酸球数が基準値内へ改善(0.44 THOU/uL、基準範囲: 0-0.57 THOU/uL)
*   2014年07月09日 (Day 190): 試験を完了。

---

**クエリ:**

患者ID: 01-701-1097

*   **クエリNo.:** 1
    *   **臨床試験結果への影響度合い:** Major
    *   **変数名と値:** AE.AESTDTC, AE.AEENDTC, AE.AEDTC, AE.AETERM
    *   **医療機関への問い合わせ文面:** 
        *   適用部位そう痒症の有害事象について、AESTDTCとAEDTCに矛盾が見られます。AESTDTCとAEDTCについて、正しい日付を教えてください。
        * 